# Numba Vectorize

## ufuncs

In [1]:
import numpy as np
import numba
import math
from numba import float64, int64

from time import time

In [2]:
def numpy_sin(a, b):
    return np.sin(a) + np.sin(b)

In [3]:
@np.vectorize
def py_sin(a, b):
    return math.sin(a) + math.sin(b)

In [4]:
@numba.vectorize([float64(float64, float64), int64(int64, int64)], target='parallel')
def numba_sin(a, b):
    return np.sin(a) + np.sin(b)

In [5]:
x = np.random.randint(0, 10, size=10_000_000)
y = np.random.randint(0, 10, size=10_000_000)
r = 10

In [6]:
x.dtype, y.dtype

(dtype('int64'), dtype('int64'))

In [7]:
%time _ = numpy_sin(x, y)
%time _ = py_sin(x, y)
%time _ = numba_sin(x, y)

CPU times: user 290 ms, sys: 40.8 ms, total: 331 ms
Wall time: 330 ms
CPU times: user 2.81 s, sys: 159 ms, total: 2.97 s
Wall time: 2.97 s
CPU times: user 523 ms, sys: 188 ms, total: 711 ms
Wall time: 110 ms


In [8]:
start = time()

for i in range(r):
    numpy_sin(x, y)

mflops = 2.0 * r * x.shape[0] / ((time() - start) * 1e6)
print("Numpy: {} MFlops/sec".format(mflops))

Numpy: 67.42008590374543 MFlops/sec


In [9]:
start = time()

for i in range(r):
    py_sin(x, y)

mflops = 2.0 * r * x.shape[0] / ((time() - start) * 1e6)
print("Python: {} MFlops/sec".format(mflops))

Python: 6.902808952432553 MFlops/sec


In [10]:
start = time()

for i in range(r):
    numba_sin(x, y)

mflops = 2.0 * r * x.shape[0] / ((time() - start) * 1e6)
print("Numba: {} MFlops/sec".format(mflops))

Numba: 175.27794731553533 MFlops/sec


## Paralelisasi

In [11]:
@np.vectorize
def prima_numpy_v(n):
    if n <= 1:
        raise ArithmeticError('n <= 1')
    if n == 2 or n == 3:
        return True
    elif n % 2 == 0:
        return False
    else:
        n_sqrt = math.ceil(math.sqrt(n))
        for i in range(3, n_sqrt+1):
            if n % i == 0:
                return False
    
    return True

In [12]:
@numba.vectorize(['boolean(int64)', 'boolean(int32)'])
def prima_numba_v(n):
    if n <= 1:
        raise ArithmeticError('n <= 1')
    if n == 2 or n == 3:
        return True
    elif n % 2 == 0:
        return False
    else:
        n_sqrt = math.ceil(math.sqrt(n))
        for i in range(3, n_sqrt+1):
            if n % i == 0:
                return False
    
    return True

In [13]:
@numba.vectorize(['boolean(int64)', 'boolean(int32)'], target='parallel')
def prima_numba_vp(n):
    if n <= 1:
        raise ArithmeticError('n <= 1')
    if n == 2 or n == 3:
        return True
    elif n % 2 == 0:
        return False
    else:
        n_sqrt = math.ceil(math.sqrt(n))
        for i in range(3, n_sqrt+1):
            if n % i == 0:
                return False
    
    return True

In [14]:
angka = np.random.randint(2, 10_000_000_000, dtype=np.int64, size=1000_000)
# %time p1 = prima_numpy_v(angka)
%time p2 = prima_numba_v(angka)
%time p3 = prima_numba_vp(angka)


CPU times: user 3.8 s, sys: 17.2 ms, total: 3.82 s
Wall time: 3.82 s
CPU times: user 7.83 s, sys: 39.8 ms, total: 7.87 s
Wall time: 1.08 s


In [15]:
angka = np.arange(2, 10_000_000, dtype=np.int32)

In [16]:
p1 = prima_numba_vp(angka)
prima = angka[p1]
prima[-10:]

array([9999889, 9999901, 9999907, 9999929, 9999931, 9999937, 9999943,
       9999971, 9999973, 9999991], dtype=int32)

In [17]:
%time _ = prima_numba_v(angka)
%time _ = prima_numba_vp(angka)

CPU times: user 2.13 s, sys: 8.58 ms, total: 2.13 s
Wall time: 2.13 s
CPU times: user 4.01 s, sys: 212 ms, total: 4.23 s
Wall time: 551 ms


## Generalized ufuncs 

Numba menawarkan `guvectorize` untuk mengenerate generalized ufuncs yang bekerja pada array input dengan dimensi yang berbeda.

In [18]:
import numpy as np
import numba
from numba import float32

### Penambahan skalar-vektor (colum/row-wise summation)

In [19]:
@numba.guvectorize([(float32[:], float32[:], float32[:])], '(m),()->(m)')
def scalar_vector_add(x, y, z):
    for i in range(x.shape[0]):
        z[i] = x[i] + y[0]

In [20]:
x = np.arange(10.0)
z = scalar_vector_add(x, 2.0)
print(x, z, sep='\n')

TypeError: ufunc 'scalar_vector_add' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

### Perkalian matriks-vector

In [21]:
@numba.guvectorize(['(f8[:, :], f8[:], f8[:])'], '(m,n), (n) -> (m)')
def mat_vec_mult(X, y, z):
    for i in range(X.shape[0]):
        w = 0.0
        for j in range(X.shape[1]):
            w += X[i,j] * y[j]
        z[i] = w

In [22]:
A = np.arange(9.0).reshape(3, 3)
x = np.array([1., 2., 3.])
b = mat_vec_mult(A, x)
print(A, x, b, sep='\n')

[[0. 1. 2.]
 [3. 4. 5.]
 [6. 7. 8.]]
[1. 2. 3.]
[ 8. 26. 44.]


### Perkalian matriks-matriks

In [23]:
@numba.guvectorize(['(f8[:, :], f8[:, :], f8[:, :])'], '(m,n), (n,k) -> (m,k)')
def mat_mult(X, Y, Z):
    for i in range(X.shape[0]):
        for j in range(Y.shape[1]):
            w = 0.0
            for k in range(X.shape[1]):
                w += X[i,k] * Y[k,j]
            
            Z[i,j] = w

In [24]:
A = np.arange(9.0).reshape(3, 3)
B = np.arange(15.0).reshape(3, 5)
C = mat_mult(A, B)
print(A, B, C, sep='\n')

[[0. 1. 2.]
 [3. 4. 5.]
 [6. 7. 8.]]
[[ 0.  1.  2.  3.  4.]
 [ 5.  6.  7.  8.  9.]
 [10. 11. 12. 13. 14.]]
[[ 25.  28.  31.  34.  37.]
 [ 70.  82.  94. 106. 118.]
 [115. 136. 157. 178. 199.]]


In [26]:
numba.guvectorize?